In [2]:
import openai
import os
import json
import time
import pandas as pd
from IPython.display import display, Markdown, HTML
import warnings
warnings.filterwarnings('ignore')

# Set your OpenAI API key here (or use environment variable)
# Option 1: Set directly (not recommended for production)
# openai.api_key = "your-api-key-here"

# Option 2: Use environment variable (recommended)
# os.environ['OPENAI_API_KEY'] = "your-api-key-here"

# Option 3: Load from .env file
#try:
#    from dotenv import load_dotenv
#    load_dotenv()
#    openai.api_key = os.getenv('OPENAI_API_KEY')
#except:
#    print("python-dotenv not available. Please set API key manually.")

# Option 3: Load from Google colab keys
openai_key = userdata.get('openai_key')


# Initialize OpenAI client
client = openai.OpenAI(api_key=openai_key)

print("✅ Setup complete! Ready for drilling and workover AI examples.")

✅ Setup complete! Ready for drilling and workover AI examples.


In [4]:
def display_response(title, prompt, response, color="blue"):
    """Display API responses in a formatted way"""
    html = f"""
    <div style="border-left: 4px solid {color}; padding-left: 15px; margin: 10px 0;">
        <h4 style="color: {color}; margin-bottom: 10px;">{title}</h4>
        <p><strong>Prompt:</strong> {prompt}</p>
        <p><strong>Response:</strong></p>
        <div style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; white-space: pre-wrap;">
{response}
        </div>
    </div>
    """
    display(HTML(html))

In [5]:
"""
Basic completion example - AI as drilling consultant
"""

def basic_drilling_consultation(question, model="gpt-4o-mini"):
    """Basic drilling consultation using OpenAI"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": question}],
            max_tokens=300,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Example questions
drilling_questions = [
    "What are the main causes of stuck pipe during drilling operations?",
    "Explain the difference between rotary drilling and directional drilling.",
    "What safety precautions should be taken during a workover operation?"
]

print("🔧 Basic Drilling Consultation Examples")
print("=" * 50)

for i, question in enumerate(drilling_questions, 1):
    response = basic_drilling_consultation(question)
    display_response(f"Drilling Consultation {i}", question, response, "darkblue")

🔧 Basic Drilling Consultation Examples


In [6]:
"""
System messages to create a specialized drilling engineer AI persona/system prompt
"""

class DrillingEngineerAI:
    def __init__(self, model="gpt-4o-mini"):
        self.model = model
        self.system_prompt = """
        You are a Senior Drilling Engineer with 20+ years of experience in oil and gas operations.
        You specialize in:
        - Well planning and drilling optimization
        - Workover and completion operations
        - Drilling fluid management
        - Wellbore stability and formation evaluation
        - Safety protocols and risk management

        Provide technical, accurate responses with practical recommendations.
        Include relevant calculations when appropriate and always emphasize safety.
        Use industry-standard terminology and reference common practices.
        """

    def consult(self, query, max_tokens=400):
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": query}
                ],
                max_tokens=max_tokens,
                temperature=0.5
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error: {str(e)}"

# Initialize drilling engineer AI
drilling_ai = DrillingEngineerAI()

# Test with drilling-specific scenarios
drilling_scenarios = [
    "We're experiencing high torque and drag at 8,500 ft TVD in a horizontal Bakken well. What could be causing this and what are your recommendations?",
    "Design a mud program for drilling through a high-pressure zone at 12,000 ft with potential H2S contamination.",
    "Our production has declined by 40% over 6 months. Recommend a workover strategy for this vertical well in the Eagle Ford."
]

print("\n👨‍🔬 Drilling Engineer AI Consultations")
print("=" * 50)

for i, scenario in enumerate(drilling_scenarios, 1):
    response = drilling_ai.consult(scenario)
    display_response(f"Engineering Consultation {i}", scenario, response, "darkgreen")


👨‍🔬 Drilling Engineer AI Consultations


In [7]:
"""
Few-shot prompting for drilling problem classification
"""

def classify_drilling_problems(examples, new_problem):
    """Classify drilling problems using few-shot learning"""

    # Build prompt with examples
    prompt = "Classify drilling problems into categories based on these examples:\n\n"

    for example in examples:
        prompt += f"Problem: {example['problem']}\n"
        prompt += f"Category: {example['category']}\n"
        prompt += f"Severity: {example['severity']}\n\n"

    prompt += f"Problem: {new_problem}\n"
    prompt += "Category:"

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,
            temperature=0.2  # Low temp for consistent classification
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Training examples for drilling problem classification
problem_examples = [
    {
        "problem": "Drill string is stuck and cannot rotate or move up/down",
        "category": "Mechanical Sticking",
        "severity": "High"
    },
    {
        "problem": "Mud returns are not reaching surface, total loss circulation",
        "category": "Circulation Loss",
        "severity": "High"
    },
    {
        "problem": "Drilling fluid density increased unexpectedly during circulation",
        "category": "Formation Contamination",
        "severity": "Medium"
    },
    {
        "problem": "Rate of penetration dropped significantly with same parameters",
        "category": "Bit Performance",
        "severity": "Low"
    }
]

# New problems to classify
new_problems = [
    "Wellbore pressure increased rapidly, BOP activated automatically",
    "Casing cannot be run to planned depth, encountering obstruction",
    "Drill cuttings show signs of hydrocarbon influx"
]

print("\n🔍 Drilling Problem Classification (Few-Shot Learning)")
print("=" * 60)

for i, problem in enumerate(new_problems, 1):
    classification = classify_drilling_problems(problem_examples, problem)
    display_response(f"Problem Classification {i}", problem, classification, "orange")


🔍 Drilling Problem Classification (Few-Shot Learning)


In [8]:
"""
Chain-of-thought prompting for complex drilling calculations
"""

def drilling_calculation_cot(problem):
    """Solve drilling calculations with step-by-step reasoning"""

    prompt = f"""
    Solve this drilling engineering problem step by step, showing all calculations and reasoning:

    Problem: {problem}

    Please:
    1. Identify what needs to be calculated
    2. List the given parameters
    3. Show the formula or method to use
    4. Perform the calculation step by step
    5. State the final answer with appropriate units
    6. Provide any safety considerations or recommendations
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.1  # Very low for accurate calculations
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Drilling calculation problems
calculation_problems = [
    """
    Calculate the hydrostatic pressure at 8,500 ft TVD using:
    - Mud weight: 9.8 ppg
    - True vertical depth: 8,500 ft
    Find the pressure in psi and compare to formation pressure of 4,200 psi.
    """,

    """
    Determine the pump pressure required for circulation with:
    - Flow rate: 400 gpm
    - Drill pipe: 5" OD, 4.276" ID, length 8,000 ft
    - Mud viscosity: 45 cp
    - Mud density: 10.2 ppg
    Calculate the pressure loss through the drill pipe.
    """,

    """
    Calculate casing setting depth for:
    - Formation pressure gradient: 0.52 psi/ft
    - Fracture pressure gradient: 0.78 psi/ft
    - Mud weight: 9.5 ppg (0.494 psi/ft)
    - Safety margin: 200 psi
    At what depth should the next casing string be set?
    """
]

print("\n🧮 Chain-of-Thought Drilling Calculations")
print("=" * 50)

for i, problem in enumerate(calculation_problems, 1):
    solution = drilling_calculation_cot(problem.strip())
    display_response(f"Calculation {i}", problem.strip(), solution, "purple")


🧮 Chain-of-Thought Drilling Calculations


In [9]:
"""
Generate structured outputs for drilling data analysis
"""

def analyze_drilling_data_structured(data_description, output_format="json"):
    """Analyze drilling data and return structured output"""

    if output_format == "json":
        prompt = f"""
        Analyze the following drilling data and return a JSON response with this exact structure:
        {{
            "well_performance": {{
                "overall_rating": "excellent/good/fair/poor",
                "efficiency_score": 0-100,
                "key_metrics": {{
                    "avg_rop": "value with units",
                    "npt_percentage": "percentage",
                    "cost_per_foot": "USD per foot"
                }}
            }},
            "issues_identified": [
                {{
                    "issue": "description",
                    "severity": "high/medium/low",
                    "recommendation": "action to take"
                }}
            ],
            "optimization_opportunities": [
                "opportunity 1",
                "opportunity 2"
            ],
            "next_actions": [
                "immediate action 1",
                "immediate action 2"
            ]
        }}

        Drilling Data: {data_description}
        """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=600,
            temperature=0.3
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Sample drilling data for analysis
drilling_data_samples = [
    """
    Well: BAKKEN-001
    - Total Depth: 15,500 ft (8,500 ft TVD, 7,000 ft horizontal)
    - Total drilling time: 18 days
    - Average ROP: 28 ft/hr (vertical), 15 ft/hr (horizontal)
    - Non-productive time: 15% (stuck pipe incidents, circulation losses)
    - Mud costs: $450,000
    - Bit runs: 8 total, average footage per bit: 1,938 ft
    - Formation: Bakken Shale
    - Completion: 32-stage fracturing planned
    """,

    """
    Workover Operation: EAGLE-FORD-205
    - Original completion: 2019
    - Current production: 45 BOD (down from 180 BOD initial)
    - Workover scope: Recompletion with new perforations
    - Time on operation: 12 days
    - Issues encountered: Parted tubing at 6,800 ft, sand production
    - Costs to date: $280,000
    - Expected production increase: 120 BOD
    """
]

print("\n📊 Structured Drilling Data Analysis")
print("=" * 45)

for i, data in enumerate(drilling_data_samples, 1):
    analysis = analyze_drilling_data_structured(data.strip())
    display_response(f"Data Analysis {i}", data.strip()[:100] + "...", analysis, "teal")


📊 Structured Drilling Data Analysis


In [10]:
"""
Multi-turn conversation with drilling context memory
"""

class ConversationalDrillingAdvisor:
    def __init__(self, model="gpt-4o-mini"):
        self.model = model
        self.conversation_history = [
            {
                "role": "system",
                "content": """You are an expert drilling advisor. Maintain context throughout our conversation
                about drilling operations, well planning, and field operations. Reference previous parts of our
                discussion when relevant. Provide practical, safety-focused advice."""
            }
        ]

    def ask(self, question):
        """Ask a question and maintain conversation context"""
        self.conversation_history.append({"role": "user", "content": question})

        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=self.conversation_history,
                max_tokens=300,
                temperature=0.6
            )

            answer = response.choices[0].message.content
            self.conversation_history.append({"role": "assistant", "content": answer})
            return answer

        except Exception as e:
            return f"Error: {str(e)}"

    def reset_conversation(self):
        """Reset conversation history"""
        self.conversation_history = self.conversation_history[:1]  # Keep only system message

# Initialize conversational advisor
advisor = ConversationalDrillingAdvisor()

# Example conversation flow
conversation_flow = [
    "I'm planning a horizontal well in the Permian Basin. What key factors should I consider?",
    "The formation pressure is expected to be 6,800 psi at 9,500 ft TVD. What mud weight would you recommend?",
    "We encountered lost circulation at 8,200 ft. How does this change our drilling plan?",
    "Should we set an intermediate casing string now, or continue drilling?",
    "What completion strategy would work best given these conditions?"
]

print("\n💬 Conversational Drilling Advisor")
print("=" * 40)

for i, question in enumerate(conversation_flow, 1):
    answer = advisor.ask(question)
    display_response(f"Conversation Turn {i}", question, answer, "indigo")


💬 Conversational Drilling Advisor


In [11]:
"""
Compare how different temperature settings affect drilling recommendations
"""

def compare_drilling_responses(prompt, temperatures=[0.1, 0.5, 0.9]):
    """Compare responses at different temperature settings"""
    results = {}

    for temp in temperatures:
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a drilling engineer providing recommendations."
                    },
                    {"role": "user", "content": prompt}
                ],
                max_tokens=200,
                temperature=temp
            )
            results[f"Temperature {temp}"] = response.choices[0].message.content
        except Exception as e:
            results[f"Temperature {temp}"] = f"Error: {str(e)}"

    return results

# Test prompt for temperature comparison
test_prompt = """
We're experiencing vibrations and erratic torque readings while drilling at 7,200 ft in a shale formation.
The bit is a 8.5" PDC with 6 hours on bottom. What could be causing this and what should we do?
"""

print("\n🌡️ Temperature Comparison for Drilling Advice")
print("=" * 50)

temp_results = compare_drilling_responses(test_prompt)

for temp_setting, response in temp_results.items():
    color = "red" if "0.1" in temp_setting else "orange" if "0.5" in temp_setting else "gold"
    display_response(temp_setting + " (Conservative → Creative)", test_prompt[:80] + "...", response, color)


🌡️ Temperature Comparison for Drilling Advice
